<a href="https://colab.research.google.com/github/AnandJL/AnandJL/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [8]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [9]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [12]:
data.duplicated().sum()

418

In [13]:
data=data.drop_duplicates(subset=['review']).reset_index(drop=True)

In [15]:
data['sentiment'].value_counts()

,count
sentiment,
positive,24884
negative,24698


In [63]:
import re
def removeHTML(text):
  text=re.sub('<[^>]*>','',text)
  return(text)

In [64]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

In [66]:
def removeStopWords(text):
  words=text.lower().split()
  filtered_words=[word for word in words if word not in stop_words]
  text=' '.join(filtered_words)
  return(text)



In [67]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lematizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [68]:
from nltk.stem import WordNetLemmatizer

def Lemmatizing(text):
    lemmatizer = WordNetLemmatizer()  # Fix the typo here
    text = lemmatizer.lemmatize(text)
    return text



In [73]:
def removeNoise(text):
  text=re.sub(r'[^a-zA-Z\s]+', '',text)
  text=re.sub(r'(\w)\1{2,}' , r'\1',text) # Removed the extra closing parenthesis
  text=re.sub(r'\s(2,)', ' ', text).strip()
  return(text)

In [74]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [75]:
CleanedText =[]

In [76]:
for i in range(len(data)):
  text = data.review[i]
  text = removeHTML(text)
  text = removeStopWords(text)
  text = Lemmatizing(text)
  text = removeNoise(text)
  CleanedText.append(text)

In [78]:
data['CleanedText'] = CleanedText

In [79]:
data.head()

,review,sentiment,CleanedText
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode h...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [80]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=0)

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(train['CleanedText'])
x_test = vectorizer.transform(test['CleanedText'])

In [83]:
from sklearn.metrics import accuracy_score

In [84]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_train,train['sentiment'])

MultinomialNB()

In [85]:
pred_y_mnb=mnb.predict(x_test)

In [87]:
print('Accuracy:',accuracy_score(test['sentiment'],pred_y_mnb))

Accuracy: 0.8557023293334678


In [89]:
from sklearn.ensemble import RandomForestClassifier # Corrected the typo from 'ensamble' to 'ensemble'
rfc=RandomForestClassifier()
rfc.fit(x_train,train['sentiment'])

RandomForestClassifier()

In [105]:
pred_y_mnb = mnb.predict(x_test)

In [107]:
print(accuracy_score(test['sentiment'],pred_y_mnb))

0.8557023293334678


In [109]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train, train['sentiment'])
pre_y_rf = rfc.predict(x_test) # Changed rf to rfc
print('Accuracy:', accuracy_score(test['sentiment'], pre_y_rf))

Accuracy: 0.8468286780276293


In [94]:
import numpy as np
def test_classifier(clf,input_text):
  input_text = removeHTML(input_text)
  input_text = removeStopWords(input_text)
  input_text = Lemmatizing(input_text)
  input_text = removeNoise(input_text)
  input_text = vectorizer.transform([input_text])
  # Use 'input_text' for prediction instead of 'text_2_vec'
  pred_y = clf.predict(input_text) # Changed line

  if pred_y == "positive":
    return "positive"
  else:
    return "negative"

In [103]:
input_text = '''covid positive'''

print('Text: "'+input_text+'",')
print('Sentiment:',test_classifier(mnb,input_text))

Text: "covid positive",
Sentiment: negative
